# Absolute Stability Examples

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import scipy.sparse as sp
import scipy.sparse.linalg as sla
import fd_tools as fd

## Introduction

When boundary conditions are involved, von Neumann analysis doesn't tell the whole story.  For standard single-step and multi-step methods, we can try to ensure that the values $\Delta t\lambda$ lie inside the absolute stability region of the method.  Often, a good first step is to numerically compute the eigenvalues.  Below is a function that builds a finite difference approximation using $2w+1$ values at each point.  In the middle of the domain, the centered difference is used, near the boundaries, the stencils are biased, but still centered as much as possible.

In [ ]:
def first_deriv_matrix(x, w):
    
    I, J, data = [], [], []
    n = len(x)
    
    # bias to the right until we can center
    p = (2*w+1)
    for i in range(w):
        I.extend(p*[i])
        J.extend(range(p))
        data.extend(fd.fd_coeff(1, x[i], x[:p]))
        
    # now fill in centered differences
    for i in range(w, n-w-1):
        I.extend(p*[i])
        J.extend(range(i-w, i+w+1))
        data.extend(fd.fd_coeff(1, x[i], x[i-w:i+w+1]))

    # bias to the left when we can't center anymore
    for i in range(n-w-1, n):
        I.extend(p*[i])
        J.extend(range(n-p, n))
        data.extend(fd.fd_coeff(1, x[i], x[n-p:]))
        
    return sp.csr_matrix((data, (I, J)))

In [ ]:
def matrix_demo():
    x = np.linspace(0, 9, 10)
    D = first_deriv_matrix(x, 2)
    print(np.round(D.toarray(), 3))
    plt.spy(D)
    
matrix_demo()

## Eigenvalues of what?

Since we'll be explicitly setting the left-most boundary value, we want to exclude the top row and left column from $D$.  (More details in class.)

In [ ]:
def plot_eigenvalues(a, nx, cfl, width):

    dx = 0.1
    dt = cfl*(dx/a)

    x = np.arange(nx)*dx

    D = first_deriv_matrix(x, width).toarray()

    D = D[1:,1:]

    scaled_eigs = -a*dt*np.linalg.eigvals(D)

    plt.figure(figsize=(10,5))
    for i in range(2):
        plt.subplot(1, 2, i+1)
        plt.plot(np.real(scaled_eigs), np.imag(scaled_eigs), '.')
        theta = np.linspace(0, 2*np.pi, 1000)
        plt.plot(np.cos(theta)-1, np.sin(theta), '--')
        if i == 0:
            plt.title('Forward Euler')
            plt.axis('equal')
        else:
            plt.xlim([-0.01, 0.01])

    plt.figure(figsize=(10,5))
    for i in range(2):
        plt.subplot(1, 2, i+1)
        plt.plot(np.real(scaled_eigs), np.imag(scaled_eigs), '.')
        xf = np.linspace(-2, 0.1, 200)
        yf = np.linspace(-2, 2, 400)
        Xf, Yf = np.meshgrid(xf, yf)
        Zf = Xf + 1j*Yf
        R = 1 + Zf + Zf**2/2
        mod_R = np.abs(R)
        plt.contour(xf, yf, mod_R, levels=[1], linestyles='dashed')
        if i == 0:
            plt.title('RK2')
            plt.axis('equal')
        else:
            plt.xlim([-0.01, 0.01])

    plt.figure(figsize=(10,5))
    for i in range(2):
        plt.subplot(1, 2, i+1)
        plt.plot(np.real(scaled_eigs), np.imag(scaled_eigs), '.')
        xf = np.linspace(-3, 0.3, 300)
        yf = np.linspace(-3, 3, 600)
        Xf, Yf = np.meshgrid(xf, yf)
        Zf = Xf + 1j*Yf
        R = 1 + Zf + Zf**2/2 + Zf**3/6 + Zf**4/24
        mod_R = np.abs(R)
        plt.contour(xf, yf, mod_R, levels=[1], linestyles='dashed')
        if i == 0:
            plt.title('RK4')
            plt.axis('equal')
        else:
            plt.xlim([-0.01, 0.01])

In [ ]:
plot_eigenvalues(1.8, 50, 2.5, 1)

In [ ]:
plot_eigenvalues(2.2, 100, 1.8, 2)

In [ ]:
plot_eigenvalues(1.8, 50, 1.5, 3)

In [ ]:
plot_eigenvalues(1.8, 50, 1.5, 4)

In [ ]:
plot_eigenvalues(1.8, 50, 1.4, 5)